# Imports

In [11]:
import pandas as pd

# Read CSVs

In [14]:
path_movies ='../Datasets/movies.csv'
path_ratings = '../Datasets/ratings.csv' 

movies = pd.read_csv(path_movies)
ratings = pd.read_csv(path_ratings)

# Treating Movies Dataset

In [15]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# Treating Ratings dataset

In [16]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
